# Rolling Planning mit Flowtime Verfahren

In [ ]:
import json
import pandas as pd
pd.set_option('display.max_rows', 40)

# Basics
import utils.basics.presenter as show
import utils.basics.converter as convert

import utils.checker as check
from ProductionDaySimulation import ProductionDaySimulation

In [ ]:
import utils.rolling_planning.init_jobs_times as rp_init
import utils.rolling_planning.procedure as rp_proced

In [ ]:
import utils.schedule_solver__arrival as ssv_a
import utils.reschedule.schedule_solver__arrival as rssv_a

## Generierung des Datensatzes für 6 Tage (Ankunft)

In [ ]:
# Datei laden
with open("data/jobshop_instances.json", "r", encoding="utf-8") as f:
    jobshop_instances = json.load(f)

instance =  jobshop_instances["instance ft10"]
df_instance = convert.jssp_dict_to_df(instance)

df_jssp, df_times = rp_init.init_jobs_with_arrivals(df_instance, 8,  u_b_mmax = 0.9)
df_jssp

In [ ]:
df_times

In [ ]:
solver_limit = 60*75 # je 75 min
solver_limit

In [ ]:
def get_schedule_filename(day: int, suffix: str = "", prefix: str = "06") -> str:
    file_template = "data/{prefix}_schedule_{day:02d}{suffix}.csv"
    if suffix:
        suffix = f"_{suffix}"
    return file_template.format(prefix=prefix,day=day, suffix=suffix)


## Params

In [ ]:
# fix
day_length = 1440
horizon_days = 3

file_template = "data/06_schedule_{day:02d}.csv"

In [ ]:
day_numb = 0
planning_end = 0

# Initialisierung (Tag 0)

In [ ]:
day_numb = 0

In [ ]:
day_start = day_length*day_numb                      # 0
day_end = day_start + day_length                     # 1440
planning_end =  day_start + horizon_days*day_length  # 4320

In [ ]:
# I)
df_jssp_curr, df_times_curr = rp_proced.filter_jobs_by_arrival_window(df_times, df_jssp, day_start, planning_end)
df_jssp_curr

In [ ]:
df_times_curr 

### (zusätzliche Jobs)

In [ ]:
"""

extra_job_prefix = 'A'

df_jssp_curr, df_times_curr = rp_init.add_extra_jobs_to_current_horizon(
    df_existing_jobs=df_jssp_curr,
    df_existing_arrivals=df_times_curr,
    df_template=df_jssp,
    start_time=day_start - 120,
    start_index=day_numb*10,
    job_prefix=extra_job_prefix,
    mean_interarrival_time=120,
    job_numb=1,
    shuffle=True
)
df_jssp_curr

"""

### Jobs vorziehen

In [ ]:
# Test
df_jssp_beforehand, df_times_beforehand = rp_init.sample_jobs_with_times_after_arrivaltime(df_jssp, df_times, 
                                                                             min_arrival_time=planning_end,
                                                                             n=2, random_state=123)
df_times_beforehand

In [ ]:
df_jssp_curr, df_times_curr = rp_init.add_beforehand_jobs_to_current_horizon(
    df_existing_jobs = df_jssp_curr,
    df_existing_times = df_times_curr,
    df_jssp = df_jssp,
    df_times = df_times,
    min_arrival_time = planning_end,
    n=3,
    random_state=23
)
df_times_curr

### Scheduling

In [ ]:
df_plan = ssv_a.solve_jssp_weighted_individual_flowtime(df_jssp_curr, df_times_curr , solver_time_limit=solver_limit, threads = 8)
df_plan

In [ ]:
df_plan.to_csv(get_schedule_filename(day=day_numb), index = False)

### show.plot_gantt_machines(df_plan)
check.check_all_constraints(df_plan)

In [ ]:
show.plot_gantt_jobs(df_plan)

### Simulation

In [ ]:
simulation = ProductionDaySimulation(df_plan, vc=0.35)
df_execution, df_undone = simulation.run(start_time = day_start, end_time=day_end)

In [ ]:
show.plot_gantt_machines(df_execution, duration_column="Simulated Processing Time")

In [ ]:
df_undone

## Day 1

In [ ]:
day_numb = 1

day_start = day_length*day_numb                      # 1440
day_end = day_start + day_length                     # 2880
planning_end =  day_start + horizon_days*day_length  # 5760

In [ ]:
# I)
df_jssp_curr, df_times_curr = rp_proced.filter_jobs_by_arrival_window(df_times, df_jssp, day_start, planning_end)
df_jssp_curr

In [ ]:
# evtl. nicht gestarte Operationen ergänzen
df_jssp_curr = rp_proced.extend_with_undone_operations(df_jssp_curr, df_undone)
df_jssp_curr

In [ ]:
df_times_curr = rp_proced.update_times_after_operation_changes(df_times, df_jssp_curr)
df_times_curr

In [ ]:
# zusätzliche Jobs (vorzeitig)
df_jssp_curr, df_times_curr = rp_init.add_beforehand_jobs_to_current_horizon(
    df_existing_jobs = df_jssp_curr,
    df_existing_times = df_times_curr,
    df_jssp = df_jssp,
    df_times = df_times,
    min_arrival_time = planning_end,
    n=3,
    random_state=23
)
df_times_curr

In [ ]:
# relevante Bedingungen aus der Simulation
df_execution_important = rp_proced.get_operations_running_into_day(df_execution, day_start)
df_execution_important

### Rescheduling

#### a) r = 0.8

In [ ]:
this_r = 0.8 # 80 % Effizienz, 20 % Stabilität

In [ ]:
df_reschedule = rssv_a.solve_jssp_bi_criteria_flowtime_deviation_with_fixed_ops(
    df_jssp = df_jssp_curr,
    df_arrivals = df_times_curr,
    df_executed=df_execution_important,
    df_original_plan=df_plan,
    r = this_r, 
    solver_time_limit = solver_limit,
    reschedule_start = day_start,
    threads = 8
)
df_reschedule

In [ ]:
df_reschedule.to_csv(get_schedule_filename(day=day_numb, suffix=f"r{this_r:.3f}"), index = False)

#### b) r = 0.2

In [ ]:
this_r = 0.2 # 20 % Effizienz, 80 % Stabilität

df_reschedule = rssv_a.solve_jssp_bi_criteria_flowtime_deviation_with_fixed_ops(
    df_jssp = df_jssp_curr,
    df_arrivals = df_times_curr,
    df_executed=df_execution_important,
    df_original_plan=df_plan,
    r = this_r, 
    solver_time_limit = solver_limit,
    reschedule_start = day_start,
    threads = 8
)
df_reschedule

In [ ]:
df_reschedule.to_csv(get_schedule_filename(day=day_numb, suffix=f"r{this_r:.3f}"), index = False)

#### c) r = 0.4

In [ ]:
this_r = 0.4

df_reschedule = rssv_a.solve_jssp_bi_criteria_flowtime_deviation_with_fixed_ops(
    df_jssp = df_jssp_curr,
    df_arrivals = df_times_curr,
    df_executed=df_execution_important,
    df_original_plan=df_plan,
    r = this_r, 
    solver_time_limit = solver_limit,
    reschedule_start = day_start,
    threads = 8
)
df_reschedule

In [ ]:
df_reschedule.to_csv(get_schedule_filename(day=day_numb, suffix=f"r{this_r:.3f}"), index = False)

In [ ]:
df_plan = df_reschedule

### Simulation

In [ ]:
simulation = ProductionDaySimulation(df_plan, vc=0.35)
df_execution, df_undone = simulation.run(start_time = day_start, end_time=day_end)

In [ ]:
show.plot_gantt_machines(df_execution, duration_column="Simulated Processing Time")

In [ ]:
df_undone

## Day 2-4

In [ ]:
last_planning_start = 4

for day_numb in range(2, last_planning_start + 1):  # Schleife von 2 bis 5
    day_start = day_length*day_numb                     
    day_end = day_start + day_length                 
    planning_end =  day_start + horizon_days*day_length
    
    # Ia aktuelles Ankunftszeitfenster
    df_jssp_curr, df_times_curr = rp_proced.filter_jobs_by_arrival_window(df_times, df_jssp, day_start, planning_end)

    # Ib Operation ergänzen, die zuvor bekannt und nicht abgeschlossen (aber evtl. nicht aktuellen Zeitfenster) 
    df_jssp_curr = rp_proced.extend_with_undone_operations(df_jssp_curr, df_undone)

    # Ic Zeiten-Dataframe aktualisieren (wegen Ib)
    df_times_curr = rp_proced.update_times_after_operation_changes(df_times, df_jssp_curr)


    # II drei zusätzliche Jobs (vorzeitig)
    df_jssp_curr, df_times_curr = rp_init.add_beforehand_jobs_to_current_horizon(
        df_existing_jobs = df_jssp_curr,
        df_existing_times = df_times_curr,
        df_jssp = df_jssp,
        df_times = df_times,
        min_arrival_time = planning_end,
        n=3,
        random_state=23
    )

    # III relevante Bedingungen aus der Simulation (Operationen, die in aktuellen Planunghorizont hineinlaufen)
    df_execution_important = rp_proced.get_operations_running_into_day(df_execution, day_start)

    # Rescheduling
    this_r = 0.4   # 40 % Effizienz, 60 % Stabilität

    df_plan = rssv_a.solve_jssp_bi_criteria_flowtime_deviation_with_fixed_ops(
        df_jssp = df_jssp_curr,
        df_arrivals = df_times_curr,
        df_executed=df_execution_important,
        df_original_plan=df_plan,
        r = this_r, 
        solver_time_limit = solver_limit,
        reschedule_start = day_start,
        threads = 8
    )
    df_plan.to_csv(get_schedule_filename(day=day_numb), index = False)

    show.plot_gantt_machines(df_plan, title = f"Gantt-Diagramm ab Tag {day_numb}")
    check.check_all_constraints(df_plan)

    # Simulation (ein Tag)
    simulation = ProductionDaySimulation(df_plan, vc=0.35)
    df_execution, df_undone = simulation.run(start_time = day_start, end_time=day_end)
    show.plot_gantt_machines(df_execution, 
                             title = f"Gantt-Diagramm für Simulationstag {day_numb}",
                             duration_column = "Simulated Processing Time")